This notebook calculates secondary parameters from the measured glider data. It uses the **`gliderParameterCalcs`** module.

# Initializing

## Required modules

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import os
import re
import gliderParameterCalcs as gpc
import quenchingFunctions as qf

import warnings

warnings.filterwarnings('ignore')

## File list

In [2]:
parentDir = '../../data/mission-netcdf/quenchingCorrected-v4/'
files = os.listdir(parentDir)
gFiles = [parentDir + ff for ff in files if os.path.isfile(parentDir+ff)]

## Mission names

In [3]:
fullnames = [ff[:-3] for ff in files if os.path.isfile(parentDir+ff)]
directions = ['east','west']
mid = [fn + '_' + dd for fn in fullnames for dd in directions]

# Making `secondaryParams-v2`

In [4]:
MLD = {}
QI = {}
Zeu = {}
Zchl = {}
PAR = {}
chlC = {}
bbC = {}
sStrength = {}
avChl = {}
intChl = {}
intChl2 = {}
intChl3 = {}
times = {}
for gf in gFiles:
    gdata = xr.open_dataset(gf)
    gid = re.split('/',gf)[-1][:-3]
    print(gid)

    ## PAR
    par = gpc.PARcalc(gdata)
    pars = {}
    for dd in ['east','west']:
        pars[dd] = np.nanmean(par[dd][0:10,:],axis = 0)    
    PAR[gid] = pars

    ## MLD
    mld = gpc.MLDCalc(gdata)
    MLD[gid] = mld    

    #QI
    density_smooth = gpc.smoothing(gdata,'sigmat',7,3)
    qi = gpc.QICalc(density_smooth, mld, gdata.zbin.data)
    QI[gid] = qi

    ## Calculating Zeu
    zeu = gpc.Zeucalc(gdata,par) ###### SHOULD BE PAR??
    Zeu[gid] = zeu

    ## Stratification strength
    ss = gpc.stratStrengthcalc_smooth(gdata)
    sStrength[gid] = ss

    try:
        ##average chl in upper 65m
        achl = gpc.avChlcalc_corr(gdata)
        avChl[gid] = achl

        ##integrated chl in upper 65m
        ichl = gpc.intChlcalc_corr_depth(gdata,65)
        intChl[gid] = ichl
        ichl2 = gpc.intChlcalc_corr_depth(gdata,100)
        intChl2[gid] = ichl2
        ichl3 = gpc.intChlcalc_corr_depth(gdata,200)
        intChl3[gid] = ichl3

    except KeyError:
        
        avChl[gid] = {k:v*np.nan for k, v in zeu.items()}
        intChl[gid] = {k:v*np.nan for k, v in zeu.items()}
        intChl2[gid] = {k:v*np.nan for k, v in zeu.items()}
        intChl3[gid] = {k:v*np.nan for k, v in zeu.items()}
        
    #time
    tt = {}
    for dd in ['east','west']:
        daytime = gdata['day_'+dd].data
        tt[dd] = np.expand_dims(np.nanmean(daytime,axis = 0)- (5./24.),axis=0)
    times[gid] = tt
    
varnames = list(Zeu.keys())
secondaryParams = xr.Dataset()
secondaryParams.coords['lonbin'] = ('lonbin',gdata['lonbin'].data)

Zeuphotic = np.ones((len(varnames)*2,297))*np.nan
Zml = np.ones((len(varnames)*2,297))*np.nan
QImld = np.ones((len(varnames)*2,297))*np.nan
par = np.ones((len(varnames)*2,297))*np.nan
stratSt = np.ones((len(varnames)*2,297))*np.nan
avchl = np.ones((len(varnames)*2,297))*np.nan
intchl = np.ones((len(varnames)*2,297))*np.nan
intchl2 = np.ones((len(varnames)*2,297))*np.nan
intchl3 = np.ones((len(varnames)*2,297))*np.nan
jtime = np.ones((len(varnames)*2,297))*np.nan
ids = []
for jj,dd in enumerate(['east','west']):
    for ii,vv in enumerate(varnames):
        Zeuphotic[ii+len(varnames)*jj,:] = Zeu[vv][dd]
        Zml[ii+len(varnames)*jj,:] = MLD[vv][dd]
        QImld[ii+len(varnames)*jj,:] = QI[vv][dd]
        par[ii+len(varnames)*jj,:] = PAR[vv][dd]
        avchl[ii+len(varnames)*jj,:] = avChl[vv][dd]
        intchl[ii+len(varnames)*jj,:] = intChl[vv][dd]
        intchl2[ii+len(varnames)*jj,:] = intChl2[vv][dd]
        intchl3[ii+len(varnames)*jj,:] = intChl3[vv][dd]
        stratSt[ii+len(varnames)*jj,:] = sStrength[vv][dd]
        jtime[ii+len(varnames)*jj,:] = times[vv][dd]
        ids += [vv+'_'+dd]

secondaryParams.coords['mission'] = ('mission',ids)
secondaryParams['Zeu'] = (('mission','lonbin'),Zeuphotic)
secondaryParams['MLD'] = (('mission','lonbin'),Zml)
secondaryParams['QI'] = (('mission','lonbin'),QImld)
secondaryParams['PAR'] = (('mission','lonbin'),par)
secondaryParams['intChl65'] = (('mission','lonbin'),intchl)
secondaryParams['intChl100'] = (('mission','lonbin'),intchl3)
secondaryParams['intChl200'] = (('mission','lonbin'),intchl3)
secondaryParams['avChl'] = (('mission','lonbin'),avchl)
secondaryParams['sStrength'] = (('mission','lonbin'),stratSt)
secondaryParams['times'] = (('mission','lonbin'),jtime)

secondaryParams.to_netcdf('secondaryParams.nc')

henry-mission18
grampus-mission1
grampus-mission2.1
grampus-mission2
grampus-mission3
grampus-mission4
grampus-mission5
grampus-mission6
grampus-mission7
grampus-mission8
henry-mission1
henry-mission2
henry-mission3
henry-mission4
henry-mission5
henry-mission6
henry-mission7.1
henry-mission7
henry-mission8
henry-mission9
henry-mission10
henry-mission11
henry-mission12
henry-mission13
henry-mission14
henry-mission15
henry-mission16
henry-mission17
